# Risk Atlas Nexus - Simple Risk Identification Demo

This notebook demonstrates how to use the Risk Atlas Nexus library to identify AI risks associated with a prompt and propose a safer prompt rewrite.

Based on the example from: https://github.com/IBM/risk-atlas-nexus/blob/main/docs/examples/notebooks/risk_identification.ipynb

In [23]:
# Add Mellea integration for risk mitigation
from mellea import generative, start_session

# Initialize Mellea session
m = start_session()
print("✓ Mellea session started")

@generative
def rewrite_prompt_for_risk_mitigation(original_prompt: str, identified_risk: str) -> str:
    """Rewrite the given prompt to minimize the specified AI risk while maintaining core intent."""

print("✓ Mellea risk mitigation method defined")

✓ Mellea session started
✓ Mellea risk mitigation method defined


In [24]:
# Example: Test the Mellea risk mitigation function
print("🧪 Testing Mellea risk mitigation...")

# Example usage
original_prompt_text = "Generate personalized customer service responses with recommendations."
example_risk_text = "Data bias in personalized recommendations"

try:
    print(f"Original prompt: {original_prompt_text}")
    print(f"Risk to mitigate: {example_risk_text}")
    print("🔧 Generating safer prompt...")
    
    # Call the Mellea generative function with correct pattern (session first, then named params)
    safer_prompt = rewrite_prompt_for_risk_mitigation(
        m, 
        original_prompt=original_prompt_text, 
        identified_risk=example_risk_text
    )
    
    print(f"✅ Rewritten prompt: {safer_prompt}")
    
except Exception as e:
    print(f"❌ Error testing Mellea: {e}")
    import traceback
    traceback.print_exc()

🧪 Testing Mellea risk mitigation...
Original prompt: Generate personalized customer service responses with recommendations.
Risk to mitigate: Data bias in personalized recommendations
🔧 Generating safer prompt...
✅ Rewritten prompt: Craft individualized customer service replies, ensuring fairness and avoiding data bias in suggestions.


## 1. Setup and Imports

In [25]:
# Install risk-atlas-nexus if not already installed
import subprocess
import sys

try:
    import risk_atlas_nexus
    print("✓ risk-atlas-nexus already installed")
except ImportError:
    print("Installing risk-atlas-nexus...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "risk-atlas-nexus[ollama]"])
    print("✓ risk-atlas-nexus installed")

✓ risk-atlas-nexus already installed


In [26]:
# Import required modules
import os
from dotenv import load_dotenv

# Risk Atlas Nexus imports
from risk_atlas_nexus.blocks.inference import (
    OllamaInferenceEngine,
    RITSInferenceEngine,
    WMLInferenceEngine
)
from risk_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    OllamaInferenceEngineParams,
    RITSInferenceEngineParams,
    WMLInferenceEngineParams
)
from risk_atlas_nexus.library import RiskAtlasNexus

# Load environment variables
load_dotenv()

print("✓ All imports successful")

✓ All imports successful


## 2. Setup Inference Engine

Choose and configure an inference engine. We'll try multiple options with fallbacks.

In [28]:
def setup_inference_engine():
    """Try to setup an inference engine with fallbacks."""
    
    # Try Ollama first (local)
    try:
        ollama_url = os.getenv("OLLAMA_API_URL", "http://localhost:11434")
        inference_engine = OllamaInferenceEngine(
            model_name_or_path="granite3.2:8b",
            credentials=InferenceEngineCredentials(api_url=ollama_url),
            parameters=OllamaInferenceEngineParams(
                num_predict=1000,
                num_ctx=8192,
                temperature=0,
                repeat_penalty=1
            )
        )
        print("✓ Using Ollama inference engine with Granite 3.2 8B")
        return inference_engine
    except Exception as e:
        print(f"⚠️ Ollama setup failed: {e}")
       
    print("❌ No inference engine could be configured")
    print("Please ensure one of the following is available:")
    print("  - Ollama running locally at http://localhost:11434")
    return None

# Setup the inference engine
inference_engine = setup_inference_engine()

[2025-09-11 15:19:45:795] - INFO - RiskAtlasNexus - OLLAMA inference engine will execute requests on the server at http://localhost:11434.
[2025-09-11 15:19:45:827] - INFO - RiskAtlasNexus - Created OLLAMA inference engine.


✓ Using Ollama inference engine with Granite 3.2 8B


## 3. Initialize Risk Atlas Nexus

In [29]:
# Initialize the Risk Atlas Nexus library
try:
    risk_atlas_nexus = RiskAtlasNexus()
    print("✓ Risk Atlas Nexus library initialized successfully")
except Exception as e:
    print(f"❌ Failed to initialize Risk Atlas Nexus: {e}")
    risk_atlas_nexus = None

[2025-09-11 15:19:51:35] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


✓ Risk Atlas Nexus library initialized successfully


## 4. Define Use Case and Identify Risks

We'll identify AI risks for a customer service use case involving personalized responses and recommendations.

In [30]:
# Define the use case
prompt =  """Summarize the meeting transcript into a concise paragraph of main points."""
print("🎯 Prompt:")
print(f"   {prompt}")
print()

# Identify risks if both components are available
if risk_atlas_nexus is not None and inference_engine is not None:
    try:
        print("🔍 Identifying AI risks...")
        
        risks = risk_atlas_nexus.identify_risks_from_usecases(
            usecases=[usecase],
            inference_engine=inference_engine,
            taxonomy="ibm-risk-atlas",
            max_risk=5
        )
        
        print(f"✓ Risk identification completed")
        print(f"📊 Found {len(risks)} potential risks")
        
    except Exception as e:
        print(f"❌ Risk identification failed: {e}")
        risks = None
else:
    print("⚠️ Cannot identify risks - missing required components")
    if risk_atlas_nexus is None:
        print("   - Risk Atlas Nexus not initialized")
    if inference_engine is None:
        print("   - Inference engine not configured")
    risks = None

🎯 Prompt:
   Summarize the meeting transcript into a concise paragraph of main points.

🔍 Identifying AI risks...


Inferring with OLLAMA:   0%|          | 0/1 [00:00<?, ?it/s]

✓ Risk identification completed
📊 Found 1 potential risks


In [32]:
# Display the identified risks
if risks is not None and len(risks) > 0:
    # risks is List[List[Risk]] - get the first (and only) list of risks for our single use case
    use_case_risks = risks[0] if risks else []
    
    if len(use_case_risks) > 0:
        print("🚨 Identified AI Risks:")
        print("=" * 50)
        
        for i, risk in enumerate(use_case_risks, 1):
            # risk is a Risk object, not a dictionary
            print(f"\n{i}. Risk: {str(risk)}")
            
            # Try to access risk attributes if available
            try:
                if hasattr(risk, 'name'):
                    print(f"   Name: {risk.name}")
                if hasattr(risk, 'description'):
                    print(f"   Description: {risk.description}")
                if hasattr(risk, 'category'):
                    print(f"   Category: {risk.category}")
                if hasattr(risk, 'severity'):
                    print(f"   Severity: {risk.severity}")
                if hasattr(risk, 'mitigation'):
                    print(f"   Mitigation: {risk.mitigation}")
            except Exception as e:
                print(f"   (Risk details not accessible: {e})")
    else:
        print("✓ No specific risks identified for this use case")
        
elif risks is not None and len(risks) == 0:
    print("✓ No specific risks identified for this use case")
else:
    print("⚠️ Risk analysis was not performed")
    
   

🚨 Identified AI Risks:

1. Risk: id='atlas-incorrect-risk-testing' name='Incorrect risk testing' description='A metric selected to measure or track a risk is incorrectly selected, incompletely measuring the risk, or measuring the wrong risk for the given context.' url='https://www.ibm.com/docs/en/watsonx/saas?topic=SSYOK8/wsj/ai-risk-atlas/incorrect-risk-testing.html' dateCreated=datetime.date(2024, 9, 24) dateModified=datetime.date(2025, 4, 28) isDetectedBy=None hasRelatedAction=None isDefinedByTaxonomy='ibm-risk-atlas' isPartOf='ibm-risk-atlas-governance' closeMatch=None exactMatch=None broadMatch=['nist-value-chain-and-component-integration'] narrowMatch=None relatedMatch=['credo-risk-032', 'mit-ai-causal-risk-entity-human', 'mit-ai-causal-risk-intent-unintentional', 'mit-ai-causal-risk-timing-post-deployment', 'mit-ai-risk-subdomain-6.5'] detectsRiskConcept=None tag='incorrect-risk-testing' type='non-technical' phase=None descriptor='amplified by generative AI' concern='If the metr

In [33]:
original_prompt_text = prompt

# Display the identified risks
if risks is not None and len(risks) > 0:
    # risks is List[List[Risk]] - get the first (and only) list of risks for our single use case
    use_case_risks = risks[0] if risks else []
    original_prompt = usecase
    if len(use_case_risks) > 0:
        print("🚨 Identified AI Risks:")
        print("=" * 50)
        
        for i, risk in enumerate(use_case_risks, 1):

            try:
                # risk is a Risk object, not a dictionary
                print(f"Original prompt: {original_prompt_text}")
                print(f"Risk to mitigate: {str(risk.name)}")
                
                # Call the Mellea generative function with correct pattern (session first, then named params)
                safer_prompt = rewrite_prompt_for_risk_mitigation(
                    m, 
                    original_prompt=original_prompt_text, 
                    identified_risk=risk.description
                )
                
                print(f"✅ Rewritten prompt: {safer_prompt}")
            except Exception as e:
                print(f"❌ Error testing Mellea: {e}")
                import traceback
                traceback.print_exc()
            
    else:
        print("✓ No specific risks identified for this use case")

   

🚨 Identified AI Risks:
Original prompt: Summarize the meeting transcript into a concise paragraph of main points.
Risk to mitigate: Incorrect risk testing
✅ Rewritten prompt: Summarize the meeting transcript focusing on main points, ensuring the chosen metric accurately reflects and measures the relevant risk in this context.
Original prompt: Summarize the meeting transcript into a concise paragraph of main points.
Risk to mitigate: Confidential data in prompt
✅ Rewritten prompt: Summarize the meeting transcript, excluding any confidential information, into a concise paragraph of main points.
Original prompt: Summarize the meeting transcript into a concise paragraph of main points.
Risk to mitigate: Prompt leaking
✅ Rewritten prompt: Summarize the meeting transcript, focusing on key topics and actions, without revealing internal AI communication guidelines.
Original prompt: Summarize the meeting transcript into a concise paragraph of main points.
Risk to mitigate: Hallucination
✅ Rewri